In [1]:
import numpy as np
import pandas as pd

In [2]:
df_clean2 = pd.read_pickle('df_clean2.pkl')
df_clean2.keys()

Index(['DateId', 'MemberId', 'OrderGroupCode', 'TrackSourceTypeDef',
       'TrackDeviceTypeDef', 'PayProfileTypeDef', 'SalesOrderSlaveId',
       'SalePageId', 'IsMajor', 'IsGift', 'IsSalePageGift', 'Quantity',
       'UnitPrice', 'PromotionDiscount', 'ECouponId', 'ECouponDiscount',
       'SalesOrderSlaveTotalPayment', 'SalesOrderSlaveDateTime',
       'SalesOrderReceiverId', 'City', 'District', 'ZipCode', 'StoreName',
       'StoreId', 'DeliverTypeDef', 'StatusDef', 'ReturnGoodsDateId',
       'CauseDef', 'new_DateId'],
      dtype='object')

In [3]:
dict_region = {"北部":['臺北','台北','新北','基隆','宜蘭','桃園','新竹'],
               "中部":['苗栗','臺中','台中','彰化','南投','雲林'],
               "南部":['嘉義','臺南','台南','高雄','屏東','澎湖'],
               "東部":['花蓮','台東','臺東']
              }

In [4]:
m_state = df_clean2[df_clean2['City'] == df_clean2['City']] #過濾NaN
m_state.head()

,DateId,MemberId,OrderGroupCode,TrackSourceTypeDef,TrackDeviceTypeDef,PayProfileTypeDef,SalesOrderSlaveId,SalePageId,IsMajor,IsGift,...,City,District,ZipCode,StoreName,StoreId,DeliverTypeDef,StatusDef,ReturnGoodsDateId,CauseDef,new_DateId
0,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457777,1242871,True,False,...,嘉義縣,水上鄉,608,中庄門市,910828,SevenEleven,Finish,NaN,NaN,2015-07-03 00:00:00
1,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457778,1242880,True,False,...,嘉義縣,水上鄉,608,中庄門市,910828,SevenEleven,Finish,NaN,NaN,2015-07-03 00:00:00
2,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457779,1242880,True,False,...,嘉義縣,水上鄉,608,中庄門市,910828,SevenEleven,Finish,NaN,NaN,2015-07-03 00:00:00
3,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457780,1175574,True,False,...,嘉義縣,水上鄉,608,中庄門市,910828,SevenEleven,Finish,NaN,NaN,2015-07-03 00:00:00
4,20150703,365787,"b""\x04?}pe$m\x7f\x85\xa0y\x81'e5\xca\xbb\xf9\x...",Web,Mobile,SevenEleven,4457791,1194488,True,False,...,桃園市,中壢區,320,復華門市,114071,SevenEleven,Finish,NaN,NaN,2015-07-03 00:00:00


In [5]:
def get_keys(d, seri_state):
    result = list()
    for state in seri_state:
        locate_flag = 0
        for key, value in d.items():
            for a in value:
                if a in state:
                    result.append(key)
                    locate_flag = 1
                    break
            if locate_flag == 1:
                break
        if locate_flag == 0:
            result.append('not_define')
    return result                    

In [6]:
get_keys(dict_region, ['桃園'])

['北部']

In [7]:
m_state['Area'] = get_keys(dict_region, m_state['City'])

In [8]:
m_state = m_state[m_state['Area'] != 'not_define']

In [9]:
# 取出 "地區、城市、購買數量、物品編號、時間"
a0 = pd.Series(m_state['new_DateId'])
a1 = pd.Series(m_state['SalePageId'])
a2 = pd.Series(m_state['Quantity'])
a3 = pd.Series(m_state['City'])
a4 = pd.Series(m_state['Area'])

data = {'new_DateId':a0, 'SalePageId':a1, 'Quantity':a2, 'City':a3, 'Area':a4}
item_loc = pd.DataFrame(data)
item_loc.head()

,Area,City,Quantity,SalePageId,new_DateId
0,南部,嘉義縣,1,1242871,2015-07-03 00:00:00
1,南部,嘉義縣,1,1242880,2015-07-03 00:00:00
2,南部,嘉義縣,1,1242880,2015-07-03 00:00:00
3,南部,嘉義縣,1,1175574,2015-07-03 00:00:00
4,北部,桃園市,1,1194488,2015-07-03 00:00:00


In [10]:
city_num = item_loc.groupby(by='City')['Quantity'].sum()
city_num

City
南投縣      74939
台中市     815918
台北市     705464
台南市     407470
台東縣      35217
嘉義市      55276
嘉義縣      68262
基隆市      89367
宜蘭縣      87111
屏東縣     125897
彰化縣     201898
新北市    1196609
新竹市     148414
新竹縣     129924
桃園市     599553
澎湖縣      20190
花蓮縣      69475
苗栗縣     108647
雲林縣      88407
高雄市     580556
Name: Quantity, dtype: int64

In [11]:
name = []
s0 = []
s1 = []
for i in range(len(city_num)):
    item_sum = ((item_loc[item_loc['City'].str.contains((city_num.keys())[i])==True]).groupby(by='SalePageId')['Quantity'].sum()).sort_values(ascending=False).head()
    for j in range(0,5):
        name.append((city_num.keys())[i])
    s0.append(pd.Series(item_sum.keys()))
    s1.append(pd.Series(np.array(item_sum)))
b2 = pd.Series(name)
b1 = pd.Series(s1)
b0 = pd.Series(s0)

In [12]:
t0 = []
t1 = []
for i in range((len(b0))):
    t0 = np.hstack((t0, np.array(b0[i])))
    t1 = np.hstack((t1, np.array(b1[i])))
s0 = pd.Series(t0)
s1 = pd.Series(t1)

In [13]:
# 找出每個縣市最喜歡購買前五名的物品
dataa = {'City':b2, 'ItemId':s0, 'Quantity':s1}
item_num_loc = pd.DataFrame(dataa)
item_num_loc.iloc[0:10]

,City,ItemId,Quantity
0,南投縣,1830694.0,393.0
1,南投縣,2336461.0,384.0
2,南投縣,2294453.0,317.0
3,南投縣,3830498.0,275.0
4,南投縣,1119387.0,271.0
5,台中市,2336461.0,4218.0
6,台中市,2294453.0,4018.0
7,台中市,1830694.0,3737.0
8,台中市,1119387.0,3399.0
9,台中市,3830498.0,3063.0


In [16]:
name = []
item_year = []
item_month = []
s0 = []
s1 = []
year = ['5', '6', '7', '8']
month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for i in range(len(city_num)):
    rr = (item_loc[item_loc['City'].str.contains(city_num.keys()[i])==True]).sort_values(['new_DateId'], ascending=True)
    for q in range(len(year)):
        y = year[q]
        for r in range(len(month)):
            m = month[r]
            first_day_of_month = pd.Timestamp('201'+y+'-'+m+'-01')
            last_day_of_month = pd.Timestamp('201'+y+'-'+m+'-01') + pd.tseries.offsets.MonthEnd()
            item_sum =(rr[(rr['new_DateId'] > first_day_of_month) & (rr['new_DateId'] < last_day_of_month)].groupby(by='SalePageId')['Quantity'].sum()).sort_values(ascending=False).head()
            for j in range(0,5):
                name.append((city_num.keys())[i])
                item_year.append(('201'+y))
                item_month.append(m)
            s0.append(pd.Series(item_sum.keys()))
            s1.append(pd.Series(np.array(item_sum)))
b4 = pd.Series(item_year)
b3 = pd.Series(item_month)
b2 = pd.Series(name)
b1 = pd.Series(s1)
b0 = pd.Series(s0)

In [17]:
len(b0), len(b1), len(b2), len(b3), len(b4)

(960, 960, 4800, 4800, 4800)

In [20]:
empty_list = []
value_list = []
value_list1 = []
b2_after = []
b3_after = []
b4_after = []
for i in range(len(b0)):
    gg = np.array(b0[i])
    if len(gg) < 4:
        empty_list.append(i)
    if len(gg) > 4:
        value_list.append(b0[i])
        value_list1.append(b1[i])
        for j in range(0,5):
            b2_after.append(b2[(i*5+j)])
            b3_after.append(b3[(i*5+j)])
            b4_after.append(b4[(i*5+j)])           

In [21]:
t0 = []
t1 = []
for i in range(len(value_list)):
    t0 = np.hstack((t0, np.array(value_list[i])))
    t1 = np.hstack((t1, np.array(value_list1[i])))
s0 = pd.Series(t0)
s1 = pd.Series(t1)
s2 = pd.Series(b2_after)
s3 = pd.Series(b3_after)
s4 = pd.Series(b4_after)

In [22]:
dataaa = {'ItemId':s0, 'Quantity':s1, 'City':s2, 'month':s3, 'year': s4}
item_num_loc_ = pd.DataFrame(dataaa)

In [23]:
item_num_loc_.head()

,City,ItemId,Quantity,month,year
0,南投縣,1162433.0,6.0,05,2015
1,南投縣,1162454.0,5.0,05,2015
2,南投縣,1175642.0,3.0,05,2015
3,南投縣,1162471.0,3.0,05,2015
4,南投縣,1119583.0,3.0,05,2015


In [24]:
i = 0
kk = item_num_loc_[item_num_loc_['City'].str.contains((city_num.keys())[i])==True]
kk

,City,ItemId,Quantity,month,year
0,南投縣,1162433.0,6.0,05,2015
1,南投縣,1162454.0,5.0,05,2015
2,南投縣,1175642.0,3.0,05,2015
3,南投縣,1162471.0,3.0,05,2015
4,南投縣,1119583.0,3.0,05,2015
5,南投縣,1194488.0,19.0,06,2015
6,南投縣,1175574.0,13.0,06,2015
7,南投縣,1194536.0,12.0,06,2015
8,南投縣,1119555.0,12.0,06,2015
9,南投縣,1194545.0,12.0,06,2015


In [26]:
ItemId_top = []
for i in range(int(len(kk)/5)):
    ItemId_top.append((kk.iloc[[(i*5)]])['ItemId'][(i*5)])

In [27]:
ItemId_top[0:100]

[1162433.0,
 1194488.0,
 1270622.0,
 1348360.0,
 1460234.0,
 1504091.0,
 1504091.0,
 1597130.0,
 1783258.0,
 1910591.0,
 1830694.0,
 1830694.0,
 1119387.0,
 1119387.0,
 2294453.0,
 2336461.0,
 2294453.0,
 2551598.0,
 2676872.0,
 2808947.0,
 2855860.0,
 2896269.0,
 3048814.0,
 3138192.0,
 3217536.0,
 3354289.0,
 3422565.0,
 3512392.0,
 3512392.0,
 3691450.0,
 3830498.0,
 3892255.0,
 3837188.0,
 4116152.0,
 4129759.0,
 4242366.0]

##### 看物品編號對應的商品，比較容易看出關係

In [29]:
raw_data = pd.read_csv('../../91APPdataset/Orders.csv')
df = pd.read_csv('../../91APPdataset/Orders.csv', header=0, sep=',')

/home/kevin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
a = ItemId_top[:17]

In [34]:
i = 16
aaa = raw_data.loc[raw_data['SalePageId'] == a[i]]
aaa

,DateId,MemberId,OrderGroupCode,TrackSourceTypeDef,TrackDeviceTypeDef,PayProfileTypeDef,SalesOrderSlaveId,SalePageId,IsMajor,IsGift,...,SalesOrderReceiverId,City,District,ZipCode,StoreName,StoreId,DeliverTypeDef,StatusDef,ReturnGoodsDateId,CauseDef
82148,20170329,2530041,b'X\xe7\xfaz\x004\xd3\xec\x8fjA\x0c\x0e\xcad\x...,Web,PC,Family,31992586,2294453,True,False,...,10991630,新北市,板橋區,220.0,全家板橋華江店,176,Family,Finish,NaN,NaN
82269,20170329,1604570,"b'V_\xf9\xf1\x80,Y\xac<\x03\n\xe5\xe1\x1e\x0e\...",AndroidApp,Mobile,Family,31993150,2294453,True,False,...,10991852,高雄市,仁武區,814.0,全家仁武永仁店,11818,Family,Finish,NaN,NaN
84532,20170317,1992782,b'\x04]^.\x04\x8b\xcc*\x0f\x8b\x05\x9f\xd4\x08...,Web,Mobile,SevenEleven,31236228,2294453,True,False,...,10721928,彰化縣,員林市,510.0,金東山門市,931711,SevenEleven,Finish,NaN,NaN
84533,20170317,1992782,b'\x04]^.\x04\x8b\xcc*\x0f\x8b\x05\x9f\xd4\x08...,Web,Mobile,SevenEleven,31236229,2294453,True,False,...,10721928,彰化縣,員林市,510.0,金東山門市,931711,SevenEleven,Finish,NaN,NaN
84681,20170317,3003851,b'\x8c\x85d\x07X\xe6\x9c\x0ch\x14\xc3T\xf7\x92...,iOSApp,Mobile,CreditCardOnce,31239693,2294453,True,False,...,10723423,桃園市,桃園區,330.0,NaN,NaN,Home,Finish,NaN,NaN
85561,20170317,1909776,b'K \xfc\xca\xc6\x0c6\xdaT\x91V\x92\tzD\xc2\xa...,iOSApp,Mobile,CreditCardOnce,31224840,2294453,True,False,...,10717688,台北市,大安區,106.0,NaN,NaN,Home,Finish,20170331.0,規格不符、商品與網頁標示不清
85562,20170317,1909776,b'K \xfc\xca\xc6\x0c6\xdaT\x91V\x92\tzD\xc2\xa...,iOSApp,Mobile,CreditCardOnce,31224841,2294453,True,False,...,10717688,台北市,大安區,106.0,NaN,NaN,Home,ReturnGoods,20170331.0,規格不符、商品與網頁標示不清
87223,20170317,3834227,b'\xf3Z\xaf\x90\x1c\xba\xf1\x14\xf8\x9ci\xea\x...,Web,Mobile,SevenEleven,31226208,2294453,True,False,...,10718164,新北市,中和區,235.0,漳和門市,152006,SevenEleven,Finish,NaN,NaN
87224,20170317,3834227,b'\xf3Z\xaf\x90\x1c\xba\xf1\x14\xf8\x9ci\xea\x...,Web,Mobile,SevenEleven,31226209,2294453,True,False,...,10718164,新北市,中和區,235.0,漳和門市,152006,SevenEleven,Finish,NaN,NaN
87332,20170317,2593865,"b""n\xa4\x01\x1b4J2\xa4\x05'6\xb6\x8f\x89E\xf8\...",AndroidApp,Mobile,Family,31227539,2294453,True,False,...,10718641,桃園市,中壢區,320.0,全家中壢龍航店,9361,Family,Cancel,NaN,NaN


##### 想觀察購買物品的數量及種類是否會隨天氣而有所變化
##### Step1 : 做每個城市每天購買的物品最多的
##### Step2 : 與天氣資訊做結合

In [35]:
name = []
item_year = []
item_month =[]
item_day =[]
s0 = []
s1 = []
year = ['5','6','7','8']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
day = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']

In [46]:
for i in range(len(city_num)): #得到程式照日期排的資訊
    rr = (item_loc[item_loc['City'].str.contains((city_num.keys())[i])==True]).sort_values(['new_DateId'], ascending=True)
    for q in range(len(year)):
        y = year[q]
        for r in range(len(month)):
            m = month[r]
            for s in range((len(day)-2)): #得到某日購買項目最多的前二名
                d = day[s]
                d1 = day[s+1]
                itemm = (rr[(rr['new_DateId']>('201'+y+'-'+m+'-'+d)) & (rr['new_DateID']<('201'+y+'-'+m+'-'+d1))].groupby(by='SalePageId')['Quantity'].sum()).sort_values(ascending = False)
                item_sum = (itemm.head(2))
                item_mean = itemm.sum()
                if ((len(item_sum) > 1)): # 判斷第一和第二數量，第一大於第二將資料收集起來
                    if (item_sum.iloc[0] > item_sum.iloc[1]):
                        item_sum = item_sum.iloc[[0]]
                        name.append((city_num.keys())[i])
                        item_year.append(('201'+y))
                        item_month.append((m+d))
                        s0.append(pd.Series(item_sum.keys()))
                        ave = (np.array(item_sum)/np.array(item_mean))
                        s1.append(ave[0])
b5 = pd.Series(item_year)
b4 = pd.Series(item_month)
b3 = pd.Series(item_day)
b2 = pd.Series(name)
b1 = pd.Series(s1)
b0 = pd.Series(s0)

KeyboardInterrupt: 